# Fill RO-Crate with dummy data

* data is changed but looks similar
* file extentions are maintained

----------------------------------------------------------------- 

- all terms found in RO-Create are collected in a dictionary
- the terms get changed
- the dictionary is used to replace the terms in the RO-Crate so the same old term will also be the same new term


In [ ]:
import json
import os

def load_json(file_path):
    """Loads a JSON file and returns its contents."""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} does not exist")
    
    with open(file_path, 'r') as json_file:
        return json.load(json_file)

    
def save_json(data, output_file):
    """Saves the modified dictionary as a JSON file."""
    with open(output_file, 'w') as json_file:
        json.dump(data, json_file, indent=4)
        

def shift_letters(text):
    shifted_text = ""
    for char in text:
        if char.isalpha():  # Check if the character is a letter
            if char == 'z':
                shifted_text += 'a'  # Wrap around for lowercase 'z'
            elif char == 'Z':
                shifted_text += 'A'  # Wrap around for uppercase 'Z'
            else:
                shifted_text += chr(ord(char) + 1)  # Shift letter
        else:
            shifted_text += char  # Keep non-alphabet characters unchanged
    return shifted_text
        
def extract_values(data):
    """Recursively extracts all values from the JSON data."""
    values = []
    
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, (dict, list)):
                # Recursively extract values from nested structures
                values.extend(extract_values(value))
            else:
                # Append non-structured values
                values.append(value)
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list)):
                # Recursively extract values from lists
                values.extend(extract_values(item))
            else:
                # Append non-structured values
                values.append(item)
    else:
        # Append primitive values directly
        values.append(data)
    
    return values

def replace_values(data, value_dict):
    """Recursively replaces values in JSON data with corresponding values from value_dict."""
    if isinstance(data, dict):
        for key, value in data.items():
            if key != "@type":
                if isinstance(value, (dict, list)):
                    # Recursively process nested dictionaries or lists
                    data[key] = replace_values(value, value_dict)
                else:
                    # Replace value if it exists in valueDict
                    if value in value_dict:
                        data[key] = value_dict[value]
    elif isinstance(data, list):
        for index, item in enumerate(data):
            if isinstance(item, (dict, list)):
                # Recursively process lists
                data[index] = replace_values(item, value_dict)
            else:
                # Replace value if it exists in valueDict
                if item in value_dict:
                    data[index] = value_dict[item]
    
    return data

def replace_after_period(d):
    modified_dict = {}
    for key, value in d.items():
        if '.' in key and '.' in value:
            key_before_period, key_after_period = key.split('.', 1)
            value_before_period, _ = value.split('.', 1)
            modified_dict[key] = f"{value_before_period}.{key_after_period}"
        else:
            modified_dict[key] = value  # Keep unchanged if no period
    return modified_dict

def main(input_file, output_json):
    # Load the JSON file
    json_data = load_json(input_file)

    # Extract all values
    values = extract_values(json_data)
    
    
    print("List of all values in the JSON file:")
    
    print (set(values))
    counter = 0
    valueDict = {}
    for each in set(values):
        
        valueDict[each] = shift_letters(each)
    
    print (valueDict)
    
    valueDict = replace_after_period(valueDict)
    
    updated_data = replace_values(json_data, valueDict)
    
    save_json(updated_data, output_json)


input_json = "ro-crate-metadata.json"  # Path to your JSON file
output_json = "output_dummy.json"

main(input_json, output_json)
